In [ ]:
import sys
import csv
import os
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import sexmachine.detector as gender
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn import metrics
from pybrain.structure import SigmoidLayer
from pybrain.datasets import ClassificationDataSet
from pybrain.utilities import percentError
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules import SoftmaxLayer
from pybrain.tools.xml.networkwriter import NetworkWriter
from pybrain.tools.xml.networkreader import NetworkReader
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from  sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import learning_curve
from sklearn.metrics import roc_curve, auc ,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
%matplotlib inline 


In [12]:
def predict_gender(name):
    sex_predictor = gender.Detector(unknown_value=u"unknown",case_sensitive=False)
    first_name= name.str.split(' ').str.get(0)
    sex= first_name.apply(sex_predictor.get_gender)
    sex_dict={'female': -2, 'mostly_female': -1,'unknown':0,'mostly_male':1, 'male': 2}
    sex_code = sex.map(sex_dict).astype(int)
    return sex_code

In [ ]:
genuine_user = graphlab.SFrame('Facebook DataSet/users.csv') ##Testing Data 
fake_users =graphlab.SFrame('Facebook DataSet/fusers.csv') ## training data

In [ ]:
##genuine_user ## Printing Data
fake_users.show()

In [ ]:
x=fake_users.column_types() ##testing columns 
y = fake_users.column_types()
print x 
print y 

In [ ]:
def read_datasets():
   
    genuine_users = graphlab.SFrame.read_csv('Facebook DataSet/users.csv')
    fake_users =graphlab.SFrame.read_csv('Facebook DataSet/fusers.csv')
    x=fake_users.column_types()
    y = fake_users.column_types()
    ##print x 
    ##print y 
    x=graphlab.SFrame.append(genuine_user,fake_users)
    y=len(fake_users)*[0] + len(genuine_users)*[1]
    ##print x
       ## print y
    return x

In [7]:
def read_datasets_pandas():
    genuine_users = pd.read_csv('Facebook DataSet/users.csv') 
    fake_users = pd.read_csv('Facebook DataSet/fusers.csv') 
    x=pd.concat([genuine_users,fake_users])   
    y=len(fake_users)*[0] + len(genuine_users)*[1]
    return x,y

In [13]:
def predict_sex(name):
    sex_predictor = SM.Detector(unknown_value=u"unknown",case_sensitive=False)
    first_name = graphlab.SFrame()
    first_name= name.split(' ')
    sex=sex_predictor.get_gender(first_name[0])
    ##first_name.apply(lambda::sex_predictor.get_gender)
    sex_dict={'female': -2, 'mostly_female': -1,'unknown':0,'mostly_male':1, 'male': 2}
    sex_code =sex_dict.get(sex)
   
    return sex_code

In [23]:
def feature_selection(x):
    lang_list = list(enumerate(np.unique(x['lang'])))   
    lang_dict = { name : i for i, name in lang_list } 
    print lang_list     #Test1
    print lang_dict     #Test2
    x.loc[:,'lang_code'] = x['lang'].map( lambda x: lang_dict[x]).astype(int)    
    x.loc[:,'sex_code']=predict_gender(x['name'])
    feature_columns_to_use = ['statuses_count','followers_count','friends_count','favourites_count','listed_count','sex_code','lang_code']
    x=x.loc[:,feature_columns_to_use]
    return x

In [ ]:
predict_sex('Emma Wattson') ## Testing the gender detector


In [ ]:
def feature_extraction(x):                             ##TEST FUNCTION NOT WORKING 
    lang_list = list(enumerate(np.unique(x['lang'])))  
    lang_dict = { name : i for i, name in lang_list } 
    #x.add_column['lang_code']
    for i in x['lang']:
        x['lang_code'] = lang_dict[i]
    
    #x.loc[:,'lang_code'] = x['lang'].map( lambda x: lang_dict[x]).astype(int) 
    for i in x['name']:
        x['sex_code']=predict_sex(i)
        #print x['sex_code']
    #feature_columns_to_use = ['statuses_count','followers_count','friends_count','favourites_count','listed_count','sex_code','lang_code']
    #x=x[:,feature_columns_to_use]
    return x


In [59]:
def train(X,y):      ##Building classifier using FFNN
    ds = ClassificationDataSet( len(X.columns), 1,nb_classes=2)
    for k in range(len(X)): 
    	ds.addSample(X.iloc[k],np.array(y[k]))
    tstdata, trndata = ds.splitWithProportion( 0.20 )
    trndata._convertToOneOfMany( )
    tstdata._convertToOneOfMany( ) 
    print "Number of testing patterns: ", len(tstdata)
    print "Number of training patterns: ", len(trndata)
    print "Input and output dimensions: ", trndata.indim, trndata.outdim
    print "First sample (input, target, class):"
    print trndata['input'][0], trndata['target'][0], trndata['class'][0]
    input_size=len(X.columns)
    target_size=1
    hidden_size = 5   
    fnn=None
    
    fnn = buildNetwork( trndata.indim, hidden_size , trndata.outdim, outclass=SoftmaxLayer )
    print fnn['in']
    trainer = BackpropTrainer( fnn, dataset=trndata,momentum=0.05, learningrate=0.1 , verbose=False, weightdecay=0.01)


    trainer.trainUntilConvergence(verbose = False, validationProportion = 0.15, maxEpochs = 100, continueEpochs = 10 )
    NetworkWriter.writeToFile(fnn, 'oliv.xml')
    predictions=trainer.testOnClassData (dataset=tstdata)
    return tstdata['class'],predictions 

In [ ]:
##feature_extraction(y) ##unusable code . 
##y

In [61]:
x,y=read_datasets_pandas()
x.describe()


#len(x)


,id,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,geo_enabled,profile_use_background_image,profile_background_tile,utc_offset,protected,verified
count,2.818000e+03,2818.000000,2818.000000,2818.000000,2818.000000,2818.000000,1728.0,721.0,2760.0,489.0,1069.000000,0.0,0.0
mean,5.374889e+08,1672.198368,371.105039,395.363023,234.541164,2.818666,1.0,1.0,1.0,1.0,1478.391020,NaN,NaN
std,2.977005e+08,4884.669157,8022.631339,465.694322,1445.847248,23.480430,0.0,0.0,0.0,0.0,8108.211889,NaN,NaN
min,3.610511e+06,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,-39600.000000,NaN,NaN
25%,3.620867e+08,35.000000,17.000000,168.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN
50%,6.162253e+08,77.000000,26.000000,306.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN
75%,6.177673e+08,1087.750000,111.000000,519.000000,37.000000,1.000000,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN
max,1.391998e+09,79876.000000,408372.000000,12773.000000,44349.000000,744.000000,1.0,1.0,1.0,1.0,36000.000000,NaN,NaN


In [62]:
x = feature_selection(x)
x.columns


[(0, 'de'), (1, 'en'), (2, 'es'), (3, 'fr'), (4, 'gl'), (5, 'it'), (6, 'nl'), (7, 'tr')]
{'fr': 3, 'en': 1, 'nl': 6, 'de': 0, 'tr': 7, 'it': 5, 'gl': 4, 'es': 2}


Index([u'statuses_count', u'followers_count', u'friends_count',
       u'favourites_count', u'listed_count', u'sex_code', u'lang_code'],
      dtype='object')

In [60]:
y_test,y_pred =train(x,y)
#print y_test
#print y_pred

Number of testing patterns:  563
Number of training patterns:  2255
Input and output dimensions:  7 2
First sample (input, target, class):
[  2.03700000e+04   5.47000000e+03   2.38500000e+03   1.45000000e+02
   5.20000000e+01   2.00000000e+00   5.00000000e+00] [1 0] [ 0.]
<LinearLayer 'in'>
